 #  Byte degree 미니 프로젝트
 ## Daum 영화 제목 및 댓글 크롤링

### 문제
    - https://movie.daum.net 에 접속하여 본인이 좋아하는 영화를 선택합니다.
    (1) 메인 Tab에서 영화 제목을 크롤링 합니다.
    (2) 평점 Tab에서 영화 댓글을 크롤링 합니다.

### 조건 
    (1) 각 크롤링은 함수을 만들어 호출하는 형태로 작성한합니다. (영화 제목과 댓글 총 2개의 함수가 만들어져야 함)
    (2) 영화 제목 크롤링 함수는 인자값으로 영화 ID를, 댓글 크롤링 함수는 인자값으로 영화 ID와 최대 page 수를 받을 수 있어야 합니다. 
    (3) 댓글 크롤링 후 최종 결과는 댓글의 Value 만 크롤링 되어야 하며 불필요한 HTML 테그가 결과에 포함되어서는 안됩니다.
    (4) 만약 댓글의 전체 page 수가 함수에 인자로 지정한 최대 page 수 보다 적게 존재하는 경우는 이를 체크하여 크롤링을 멈춥니다. 
        (e.g. 함수 인자에 최대 크롤링 page를 100으로 지정했다 하더라도 실제 크롤링 대상 page가 10개만 존재한다면 크롤링 멈춤)
    (5) 모든 작성은 jupyter notebook에서 하며 수행 결과를 포함시켜 ipynb 확장자 파일로 제출합니다.

### 추가 hint
    (1) 영화의 댓글은 get 방식으로 page 별로 호출 할 수 있습니다. (page는 1부터 시작)
    (2) BeautifulSoap의 find_all을 활용하면 리스트로 반환된 결과를 len 함수를 통해 개수를 확인할 수 있습니다.

In [1]:
import requests
from bs4 import BeautifulSoup

### 영화 메인 정보 크롤링 함수 (메인 Tab)

In [130]:
def get_daum_movie_title(movie_id):  #movie_id를 받을 수 있어야 하므로 입력변수 선언
    url = 'https://movie.daum.net/moviedb/main?movieId={}'.format(movie_id)  #url 지정후 movie_id는 입력받아야하니 format 사용
    resp = requests.get(url)    #url값을 get요청후 객체에 저장 
    
    soup = BeautifulSoup(resp.text)  #객체값의 텍스트값을 스프 객체에 넣음
    
    title_list = soup.select_one('strong.tit_movie')   #strong 이라는 객체안에  tit_movie라는 클래스가 정의되있음
    if title_list:
        return title_list.get_text()          #태그안에 텍스트값이 있으면 텍스트반환
    return ''                              #텍스트값없으면 '' 출력

In [131]:

# 영화 메인 제목 크롤링 함수 호출
get_daum_movie_title('93252')



'어벤져스: 엔드게임 (2019)'

---
>- 리뷰
   - 잘해주셨습니다!
   - 타이틀이 없을 경우에 대한 예외처리도 완벽하네요!

### 영화 댓글 크롤링 함수 (평점 Tab)

In [146]:
## 영화 댓글 크롤링 함수
def get_daum_movie_comments(movie_id, start_page, end_page):  #movie_id값과 페이지 구분을 위한 두개의 입력변수 선언 
    for num in range(start_page, end_page + 1):  #페이지의 범위설정
        comments_list = [] 
        url = 'https://movie.daum.net/moviedb/grade?movieId={}&type=netizen&page={}'.format(movie_id, num)
                                                    #movieId와 페이지를 입력받기위해 사용
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)
        container = soup.find_all('p', {'class' : 'desc_review'})    #desc_review라는 클래스 안에 p객체로 댓글이있음
        if start_page < 1 or len(container) == 0:       #페이지의 범위가 벗어나면 출력하기위해 사용
            return '페이지가 존재하지 않습니다.'           
        for comments in container:           
            comments = comments.text        #댓글변수에 댓글 텍스트값넣고
            comments_list.append(comments.strip())  #댓글텍스트값 양쪽의 빈칸 없애고 리스트값에 넣기
        print(comments_list)   #출력

In [147]:
get_daum_movie_comments(93252,1,1)

['이미 늦었지만 내가 본 최고의 영화였다 ㅎ', '비전,록키,나타샤,토니를 이제 못보다니\r추가로 캡틴도 안녕이고\r가모라는 도대체 어디간건지 \r죽은것도 아닌데 끝까지 언급이 없네\r3000만큼 사랑한 어벤저스도 이제 안녕ㅠ', '최고...', '최고', '', '참 재밌는 영화다\r 이거보고 저스티스리그보면\r저스티스리그가 참 허접하구나싶다', '인피니티 워의 스케일이 너무 컸나?\r3시간을 보며 과연 내 아까운 9,900을 냈어야 했나 생각했다. (미안 인피니티 워는 10점 줬어)', '', '엄청 재미가 있어요', '마블의 모든 것을 잘 보여준 희대의 명작!']


---
>- 리뷰
   - 잘해주셨습니다.
   - 원하는 범위의 리뷰를 모을 수 있도록 해주셨네요
   - 예외처리도 잘 넣어주셨습니다.
   - 다만 한 가지 더 추가되었으면 하는 것이 start_page와 end_page와의 비교입니다.
   - 혹여나 start_page가 end_page보다 클 수 있으니까요